<a href="https://colab.research.google.com/github/fatemafaria142/Leveraging-Large-Language-Models-for-Comprehensive-Math-Word-Problem-Solving/blob/main/Math_Word_Problem_Solving_using_Llama_2_7b_chat_hf_and_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch langchain -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━

#### **Dataset Link:**  https://huggingface.co/datasets/camel-ai/math

In [2]:
from datasets import load_dataset

# Load the dataset
instruct_tune_dataset = load_dataset("camel-ai/math")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
# Select 500 rows for training
train_subset = instruct_tune_dataset["train"].select(indices=range(500))

# Select 200 rows for testing
test_subset = instruct_tune_dataset["train"].select(indices=range(500, 700))


In [4]:
train_subset

Dataset({
    features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
    num_rows: 500
})

In [5]:
test_subset

Dataset({
    features: ['role_1', 'topic;', 'sub_topic', 'message_1', 'message_2'],
    num_rows: 200
})

In [6]:
# Check the sizes of the subsets
print("Train subset size:", len(train_subset))
print("Test subset size:", len(test_subset))

Train subset size: 500
Test subset size: 200


In [7]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("Role:", data['role_1'])
    print("Topic:", data['topic;'])
    print("Sub Topic:", data['sub_topic'])
    print("Message 1:", data['message_1'])
    print("Message 2:", data['message_2'])
    print("\n-----------------------------\n")

Data Point 1:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 5x + 3 = 28
Message 2: To solve for x, we need to isolate x on one side of the equation. 

Step 1: Subtract 3 from both sides of the equation.
5x + 3 - 3 = 28 - 3
5x = 25

Step 2: Divide both sides of the equation by 5.
5x / 5 = 25 / 5
x = 5

So, x = 5.

-----------------------------

Data Point 2:
Role: Mathematician_RoleType.ASSISTANT
Topic: Algebra
Sub Topic: Solving linear equations
Message 1: Solve for x: 2x + 5 = 9- x.
Message 2: To solve for x, we will follow these steps:

1. Add x to both sides of the equation to get all the x terms on one side:
2x + x + 5 = 9 - x + x

This simplifies to:
3x + 5 = 9

2. Subtract 5 from both sides of the equation to isolate the x terms:
3x + 5 - 5 = 9 - 5

This simplifies to:
3x = 4

3. Divide both sides of the equation by 3 to solve for x:
3x / 3 = 4 / 3

This simplifies to:
x = 4/3

So, the solution is x = 4/3.

-----

#### **Creating a Mental Health Support Prompt Using LangChain**

In [50]:
from langchain import PromptTemplate, LLMChain

def create_prompt(sample):
    # Creating prompt for large language model
    pre_prompt = """[INST] <<SYS>>\nImagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.\n"""
    prompt = pre_prompt + "My Question is: {message_1}" +"[\INST]"+"\nMath Teacher Answer: {message_2}"

    # Extract message_1 and response from the sample
    example_context = sample['message_1']
    example_response = sample['message_2']

    # Create a prompt using the template and the example
    prompt_template = PromptTemplate(template=prompt, input_variables=["message_1", "message_2"])
    single_turn_prompt = prompt_template.format(message_1=example_context, message_2=example_response)

    return single_turn_prompt


#### **Generating LangChain Prompt for a Training Subset Examples**

In [51]:
prompt = create_prompt(train_subset[0])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve for x: 5x + 3 = 28[\INST]
Math Teacher Answer: To solve for x, we need to isolate x on one side of the equation. 

Step 1: Subtract 3 from both sides of the equation.
5x + 3 - 3 = 28 - 3
5x = 25

Step 2: Divide both sides of the equation by 5.
5x / 5 = 25 / 5
x = 5

So, x = 5.


In [52]:
prompt = create_prompt(train_subset[1])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve for x: 2x + 5 = 9- x.[\INST]
Math Teacher Answer: To solve for x, we will follow these steps:

1. Add x to both sides of the equation to get all the x terms on one side:
2x + x + 5 = 9 - x + x

This simplifies to:
3x + 5 = 9

2. Subtract 5 from both sides of the equation to isolate the x terms:
3x + 5 - 5 = 9 - 5

This simplifies to:
3x = 4

3. Divide both sides of the equation by 3 to solve for x:
3x / 3 = 4 / 3

This simplifies to:
x = 4/3

So, the solution is x = 4/3.


In [53]:
prompt = create_prompt(train_subset[2])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the equation: 3x + 5 = 14.[\INST]
Math Teacher Answer: To solve the equation 3x + 5 = 14, we need to isolate the variable x. 

Step 1: Subtract 5 from both sides of the equation.
3x + 5 - 5 = 14 - 5
3x = 9

Step 2: Divide both sides of the equation by 3.
3x / 3 = 9 / 3
x = 3

So, the solution to the equation 3x + 5 = 14 is x = 3.


#### **Generating LangChain Prompt for a Testing Subset Examples**

In [54]:
prompt = create_prompt(test_subset[0])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the inequality 2x + 5 > 11, and express the solution set in interval notation.[\INST]
Math Teacher Answer: To solve the inequality 2x + 5 > 11, we'll follow these steps:

1. Subtract 5 from both sides of the inequality:
   2x + 5 - 5 > 11 - 5
   2x > 6

2. Divide both sides by 2:
   2x/2 > 6/2
   x > 3

The solution set is x > 3. In interval notation, this is expressed as (3, ∞).


In [55]:
prompt = create_prompt(test_subset[1])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the inequality: -5x + 7 < 12.[\INST]
Math Teacher Answer: To solve the inequality -5x + 7 < 12, follow these steps:

Step 1: Subtract 7 from both sides of the inequality.
-5x + 7 - 7 < 12 - 7
-5x < 5

Step 2: Divide both sides of the inequality by -5. Remember to reverse the inequality sign when dividing by a negative number.
-5x / -5 > 5 / -5
x > -1

The solution to the inequality is x > -1.


In [63]:
prompt = create_prompt(test_subset[2])
print(prompt)

[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the inequality 2x + 5 < 17, where x is a real number.[\INST]
Math Teacher Answer: To solve the inequality 2x + 5 < 17, we'll follow these steps:

1. Subtract 5 from both sides of the inequality:
2x + 5 - 5 < 17 - 5
2x < 12

2. Divide both sides of the inequality by 2:
(2x)/2 < 12/2
x < 6

So the solution to the inequality is x < 6.


### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [20]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)


bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [21]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

* **Model and Tokenizer:** https://huggingface.co/NousResearch/Llama-2-7b-chat-hf

In [22]:
mode_id = "NousResearch/Llama-2-7b-chat-hf"

In [23]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/179 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [24]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently  (without training):**
* `do_sample:`This parameter controls whether to use sampling during text generation. When do_sample is set to True, the model samples words for the next token based on their probabilities. If set to False, the model uses greedy decoding and selects the token with the highest probability at each step.
* `pad_token_id:` This parameter represents the token ID used for padding in the tokenizer. In text generation, it's common to set the padding token to the end-of-sequence token (eos_token_id) to ensure that the generated text is complete.
* `top_k:` top_k controls the number of highest probability options considered during sampling. For each token, the model will limit the sampling to the top k tokens with the highest probabilities.
* **High top_k Value:** More deterministic and focused generation. The
model considers a larger number of top probability options, resulting in conservative and predictable outputs.
* **Low top_k Value:** More randomness in sampling. The model restricts the number of top probability options, leading to diverse and creative text generation, but it may result in less coherent or focused output.

* `top_p:` top_p is used in nucleus sampling, where the model considers the cumulative probability mass of the top tokens and stops sampling when it reaches a predefined threshold. It helps in preventing the model from being overly verbose and encourages diversity in the generated text.
* **High top_p Value:** Promotes diverse and varied text generation by considering a larger range of probable tokens. It's good for creating exploratory and varied outputs.
* **Low top_p Value:** Adopts a stricter threshold for selecting tokens, resulting in more conservative and focused text generation. This may lead to more deterministic outputs closely tied to the most probable tokens.
* `temperature:` temperature controls the level of randomness in the sampling process. A higher temperature (e.g., 1.0) increases randomness, making the generated text more diverse. Lower values (e.g., 0.8) make the sampling more deterministic, with the model focusing on higher probability tokens.

**Documentation Link:**
* https://python.langchain.com/docs/integrations/llms/huggingface_pipelines


In [25]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate

# Additional configuration parameters
do_sample = True
pad_token_id = tokenizer.eos_token_id
top_p = 0.8  # You can adjust this value based on your preference
temperature = 0.5  # You can adjust this value based on your preference

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipe)

# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.\n"""
prompt = pre_prompt + "My Question is: {message_1}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["message_1"])


chain = prompt | llm

message_1 = "Solve the inequality 2x + 5 < 17, where x is a real number."

print(chain.invoke({"message_1": message_1}))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


  Of course! I'm happy to help you solve the inequality 2x + 5 < 17.

To start, let's isolate the variable x by subtracting 5 from both sides of the inequality:

2x < 12

Now, we can divide both sides of the inequality by 2 to get:

x < 6

So, the solution to the inequality is x < 6.

In other words, the value of x that satisfies the inequality is less than 6.


### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [26]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [27]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [28]:
from transformers import TrainingArguments
output_model= "Llama_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=150,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [29]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt,  # This will apply the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=train_subset,      # Pass train_subset as train_dataset
  eval_dataset=test_subset         # Pass test_subset as eval_dataset
)


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [30]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.249700
20,0.903300
30,0.612000
40,0.541200
50,0.510800
60,0.448800
70,0.407200
80,0.368600
90,0.413800
100,0.323100


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=150, training_loss=0.49707754611968996, metrics={'train_runtime': 1142.4202, 'train_samples_per_second': 0.525, 'train_steps_per_second': 0.131, 'total_flos': 1.2186386694144e+16, 'train_loss': 0.49707754611968996, 'epoch': 1.74})

### **Save the model**

In [31]:
trainer.save_model("Llama_instruct_generation")

In [32]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:249: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


#### **Generate Answers After Training**

* `invoke:` call the chain on an input
* `batch:` call the chain on a list of inputs
#### **Documentation Link:**
* https://python.langchain.com/docs/expression_language/interface

# **Conversation Buffer memory**

In [33]:
# Creating prompt for large language model
pre_prompt = """[INST] <<SYS>>\nImagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.\n"""
prompt = pre_prompt + "My Question is: {context}" +"[\INST]"

prompt = PromptTemplate(template=prompt, input_variables=["context"])

In [34]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain

pipeline = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=512, do_sample=do_sample, pad_token_id=pad_token_id, top_p=top_p, temperature=temperature)
llm = HuggingFacePipeline(pipeline=pipeline)

memory = ConversationBufferMemory()
chat_llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

#### **Example: 01**

In [39]:
context ="Solve the inequality 2x - 5 > 7, where x is a real number."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the inequality 2x - 5 > 7, where x is a real number.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Of course! I'd be happy to help you solve the inequality 2x - 5 > 7.\n\nTo start, let's rewrite the inequality as:\n\n2x > 7 + 5\n\nNow, let's isolate the variable x by subtracting 5 from both sides of the inequality:\n\n2x > 12\n\nNext, let's divide both sides of the inequality by 2 to get:\n\nx > 6\n\nTherefore, the solution to the inequality 2x - 5 > 7 is x > 6."

#### **Example: 02**

In [40]:
context = "Solve the inequality 4x - 7 ≤ 5x + 3, and write the solution in interval notation."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve the inequality 4x - 7 ≤ 5x + 3, and write the solution in interval notation.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


"  Of course! I'd be happy to help you solve the inequality 4x - 7 ≤ 5x + 3.\n\nTo start, let's rewrite the inequality as:\n\n4x - 7 ≤ 5x + 3\n\nWe can simplify the left-hand side of the inequality by combining the like terms:\n\n4x - 7 = 5x\n\nNow, we can see that the inequality becomes:\n\n5x ≤ 7 + 3\n\nWe can simplify further by adding 7 and 3:\n\n5x ≤ 10\n\nNow, we can determine the solution to the inequality by finding the value of x that makes the inequality true. In interval notation, the solution is:\n\nx ≤ 2\n\nSo, the solution to the inequality 4x - 7 ≤ 5x + 3 is x ≤ 2."

#### **Example: 03**

In [41]:
context = "Solve for x: 2x-5 > 7x+3."
chat_llm_chain.predict(context=context)



> Entering new LLMChain chain...
Prompt after formatting:
[INST] <<SYS>>
Imagine yourself as a math teacher. Your objective is to solve  Algebra and Trigonometry word problems and get the correct solution. Please read the problems attentively, solve them with mathematical terminology, and give your results coherently.
My Question is: Solve for x: 2x-5 > 7x+3.[\INST]


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



> Finished chain.


'  Of course! I\'m happy to help you solve the word problem. Let\'s break it down step by step:\n\nThe problem states: "Solve for x: 2x - 5 > 7x + 3."\n\nFirst, let\'s isolate the variable x by adding 5 to both sides of the inequality:\n\n2x > 7x + 3 + 5\n\nThis simplifies to:\n\n2x > 7x + 8\n\nNow, let\'s isolate x by subtracting 7x from both sides:\n\nx > 8\n\nFinally, let\'s solve for x by dividing both sides by 2:\n\nx = 4\n\nTherefore, the value of x is 4.'

In [42]:
memory.load_memory_variables({})

{'history': 'Human: Solve the inequality 2x - 5 > 7, where x is a real number.\nAI:   Of course! I\'d be happy to help you solve the inequality 2x - 5 > 7.\n\nTo start, let\'s rewrite the inequality as:\n\n2x > 7 + 5\n\nNow, let\'s isolate the variable x by subtracting 5 from both sides of the inequality:\n\n2x > 12\n\nNext, let\'s divide both sides of the inequality by 2 to get:\n\nx > 6\n\nTherefore, the solution to the inequality 2x - 5 > 7 is x > 6.\nHuman: Solve the inequality 4x - 7 ≤ 5x + 3, and write the solution in interval notation.\nAI:   Of course! I\'d be happy to help you solve the inequality 4x - 7 ≤ 5x + 3.\n\nTo start, let\'s rewrite the inequality as:\n\n4x - 7 ≤ 5x + 3\n\nWe can simplify the left-hand side of the inequality by combining the like terms:\n\n4x - 7 = 5x\n\nNow, we can see that the inequality becomes:\n\n5x ≤ 7 + 3\n\nWe can simplify further by adding 7 and 3:\n\n5x ≤ 10\n\nNow, we can determine the solution to the inequality by finding the value of x th

In [43]:
print(memory.buffer)

Human: Solve the inequality 2x - 5 > 7, where x is a real number.
AI:   Of course! I'd be happy to help you solve the inequality 2x - 5 > 7.

To start, let's rewrite the inequality as:

2x > 7 + 5

Now, let's isolate the variable x by subtracting 5 from both sides of the inequality:

2x > 12

Next, let's divide both sides of the inequality by 2 to get:

x > 6

Therefore, the solution to the inequality 2x - 5 > 7 is x > 6.
Human: Solve the inequality 4x - 7 ≤ 5x + 3, and write the solution in interval notation.
AI:   Of course! I'd be happy to help you solve the inequality 4x - 7 ≤ 5x + 3.

To start, let's rewrite the inequality as:

4x - 7 ≤ 5x + 3

We can simplify the left-hand side of the inequality by combining the like terms:

4x - 7 = 5x

Now, we can see that the inequality becomes:

5x ≤ 7 + 3

We can simplify further by adding 7 and 3:

5x ≤ 10

Now, we can determine the solution to the inequality by finding the value of x that makes the inequality true. In interval notation, th